## Setup

### Import Libraries

In [ ]:
import numpy as np
from tifffile import imread
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter, gaussian_filter1d

### Download Data

In [ ]:
#| output: false
import owncloud
import os

if not os.path.exists('data'):
    print('Creating directory for data')
    os.mkdir('data')

if not os.path.exists('data/demoMovie.tif'):
    oc = owncloud.Client.from_public_link('https://uni-bonn.sciebo.de/s/NwtdrIg5wGdeGcB')
    oc.get_file('/', 'data/demoMovie.tif');

In this notebook we take a close look at how the brightness of a single pixel changes over time and what it tells us about underlying neural activity. We begin by visualising different frames and projections of a movie, then focus on extracting and plotting raw traces from selected pixels. From there we explore key processing steps such as neuropil correction, baseline estimation, and signal normalization using ΔF/F₀. These steps help isolate genuine activity from background noise and form the foundation for more advanced analyses later in the course.

## Flurorescence Signals from a Single Pixel.

In this section, let us look at the brightness (fluorescence) of a single pixel over time. This brightness changes when calcium levels in the cell change which is an indicator that the neuron is "active". Plotting how the fluorescence changes shows us when "activity" might have happened in that pixel. 

#### **Exercises**

**Example**: Plot the first frame as representative image.

In [ ]:
data = imread('data/demoMovie.tif')
rep_image = data[0, :, :]
plt.imshow(rep_image, cmap='gray')

**Exercise**: Plot the 116th frame as representative image.

**Exercise**: Plot the last frame as representative image.

**Example**: Plot mean projection of all the frames as representative image.

In [ ]:
data = imread('data/demoMovie.tif')
rep_image = data[:, :, :].mean(axis=0)
plt.imshow(rep_image, cmap='gray')

**Exercise**: Plot the maximum projection of all frames as representative image.

**Exercise**: Plot maximum projection from 100th to 500th frame only.

Let us consider three cells located within rectangular box defined by:

| **Cell**   | **xmin** | **xmax** | **ymin** | **ymax** |
| :--------- | :------- | :------- | :------- | :------- |
| **Cell 1** | 15       | 35       | 20       | 35       |
| **Cell 2** | 55       | 75       | 23       | 40       |
| **Cell 3** | 0        | 10       | 23       | 40       |

**Example**: Plot mean projection image cropped to Cell 1.

In [ ]:
data = imread('data/demoMovie.tif')
rep_image = data[:, 20:35, 15:35].mean(axis=0)
plt.imshow(rep_image, cmap='gray')

**Exercise**: Plot mean projection image cropped to Cell 2.

**Exercise**: Plot max projection of only the first 100 frames cropped to Cell 3.

**Example**: Select a pixel from Cell 1.

In [ ]:
rep_image = data[:, :, :].mean(axis=0)
plt.imshow(rep_image, cmap='gray')
plt.scatter(60, 30, color='r')

**Exercise**: Select a pixel from Cell 2.

**Exercise**: Select a pixel from Cell 3.

**Example**: Plot calcium trace extracted from Cell 1.

In [ ]:
rep_image = data[:, :, :].mean(axis=0)
trace = data[:, 25, 21]

plt.subplot(211)
plt.imshow(rep_image, cmap='gray')
plt.scatter(21, 25, color='r')

plt.subplot(212)
plt.plot(trace)

**Exercise**: Plot calcium trace extracted from Cell 2.

**Exercise**: Plot calcium trace extracted from Cell 3.

## Neuropil Correction.

The brightness we see in a pixel is, usually, not just from the cell we care about. It also includes light from nearby tissue called neuropil. To get a cleaner signal, we subtract a portion of this surrounding signal. This process is called neuropil correction.

#### **Exercises**

In [ ]:
data = imread('data/demoMovie.tif')
rep_image = np.mean(data, axis=0)

**Example**: Take global mean of all frames and pixels to get neuropil trace.

In [ ]:
npil_trace = np.mean(data, axis=(1,2))
plt.plot(npil_trace)

**Exercise**: Take global median of all frames and pixels to get neuropil trace.

**Exercise**: Take global mean of all frames and pixels to get neuropil trace.

**Example**: Extract neuropil trace from just outside Cell 1.

In [ ]:
plt.subplot(311)
plt.imshow(rep_image, cmap='gray')
plt.scatter(39, 40, color='r')
plt.scatter(39, 45, color='r', marker='x')

plt.subplot(312)
plt.plot(data[:, 40, 39])

plt.subplot(313)
plt.plot(data[:, 45, 39])

**Exercise**: Extract neuropil trace from just outside Cell 2.

**Exercise**: Extract neuropil trace from just outside Cell 3.

**Example**: Subtract neuropil trace from Cell 3 calcium trace with a scaling factor of 1.0.

In [ ]:
cell_trace = data[:, 25, 21]
neuropil_trace = data[:, 28, 20]
r = 1.0
F_corr = cell_trace - r*neuropil_trace

plt.subplot(211)
plt.plot(cell_trace)
plt.plot(neuropil_trace)

plt.subplot(212)
plt.plot(F_corr)

**Exercise**: Subtract neuropil trace from Cell 2 calcium trace with a scaling factor of 0.7.

**Exercise**: Subtract neuropil trace from Cell 1 calcium trace with a scaling factor of 1.0.

## Estimating Baseline Fluorescence (F0).

Even when a neuron is not active, the signal is not zero because of slow change in signal or noise. This resting level of fluorescence is called **baseline flurorescence** or **F0**.

#### **Exercises**

In [ ]:
cell1_corr_trace = data[:, 40, 39] - 0.7 * data[:, 45, 39]
cell2_corr_trace = data[:, 30, 60] - 0.7 * data[:, 33, 56]
cell3_corr_trace = data[:, 25, 21] - 0.7 * data[:, 28, 20]

**Example**: Plot mean baseline activity of Cell 1.

In [ ]:
F0 = np.mean(cell1_corr_trace)
plt.plot(cell1_corr_trace)
plt.axhline(F0, color='r')

**Exercise**: Plot median baseline activity of Cell 2.

**Exercise**: Plot 10th percentile as baseline activity of Cell 3.

**Example**: Use moving median window to estimate baseline activity of Cell 1 with window size of 501.

In [ ]:
F0 = median_filter(cell1_corr_trace, 501)
plt.plot(cell1_corr_trace)
plt.plot(F0, color='r')

**Exercise**: Use moving median window to estimate baseline activity of Cell 2 with window size of 501.

**Exercise**: Use moving median window to estimate baseline activity of Cell 1 with an even larger window size.

**Exercise**: Use moving median window to estimate baseline activity of Cell 1 with a very small window size.

## dF/F0 Normalization.

We usually want to quantify how much the fluorescence signal increases relative to that baseline so that we can compare signals from different neurons. This is done by computing **ΔF/F₀**, where:

$\Delta F/F_0 = \frac{F - F_0}{F_0}$

Here, **F** is the fluorescence at each time point, and **ΔF** (delta F) is the difference between the current signal and the baseline. The resulting ΔF/F₀ value expresses the signal change as a **fraction of the baseline**, which is useful for comparing activity levels across different cells or imaging sessions.

#### **Exercises**

**Example**: Remove median baseline activity from Cell 1.

In [ ]:
F0 = np.median(cell1_corr_trace)
df = cell1_corr_trace - F0

plt.plot(cell1_corr_trace)
plt.plot(df)

**Exercise**: Remove baseline estimated from moving median filter with an appropriate size from the activity from Cell 3.

**Exercise**: Remove baseline estimated from moving median filter with a very small size from the activity from Cell 3.

**Example**: Plot df/f0 for Cell 1.

In [ ]:
F0 = median_filter(cell1_corr_trace, 201)
dff = (cell1_corr_trace - F0) / F0
plt.plot(dff)

**Exercise**: Plot df/f0 for Cell 2.

**Exercise**: Plot df/f0 for Cell 3.